## Knowledge Base


In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("/Users/riosky/Projects/Work/Vector-Search/playground/chromadb.ipynb"))))

from dotenv import load_dotenv
load_dotenv()

from knowledge_base import KnowledgeBase

kb = KnowledgeBase()

kb.save_questions_from_file("../data/questions.csv")

2025-08-17 20:50:14 [INFO] [chromadb_client._create_collections] - 创建集合: questions, embedding_function: <knowledge_base.config.BAAIEmbeddingFunction object at 0x326141400>
2025-08-17 20:50:14 [INFO] [chromadb_client._create_collections] - 创建集合: questions 使用自定义 embedding_function: <knowledge_base.config.BAAIEmbeddingFunction object at 0x326141400>
2025-08-17 20:50:14 [INFO] [chromadb_client._create_collections] - 创建集合: knowledge_points, embedding_function: <knowledge_base.config.BAAIEmbeddingFunction object at 0x326141400>
2025-08-17 20:50:14 [INFO] [chromadb_client._create_collections] - 创建集合: knowledge_points 使用自定义 embedding_function: <knowledge_base.config.BAAIEmbeddingFunction object at 0x326141400>
2025-08-17 20:50:14 [INFO] [utils.load_data] - DataFrame loaded from: ../data/questions.csv
2025-08-17 20:50:14 [DEBUG] [sync_manager.sync_on_insert] - Syncing on insert: <knowledge_base.storage.models.Question object at 0x33aa163c0>
2025-08-17 20:50:14 [DEBUG] [chromadb_client._get_coll

In [2]:
qc = kb.chromadb_client.get_collection("questions")
qc.query(query_texts=["程序的中断方式"], n_results=10)

2025-08-17 20:50:17 [DEBUG] [chromadb_client._get_collection_with_embedding_function] - 为集合 questions 重新应用自定义 embedding function


{'ids': [['6babab39-e2be-5bc5-b976-73ecd6c6470e',
   'c7476bfd-4b46-5324-bc93-e8fc13f6037a',
   '8f0ee14f-fc58-5ff1-8924-8d0bd3705d4c',
   '78c4b50a-1730-521b-80c4-50845f8a20b3',
   '6a7eb0f7-2d50-5fde-9d30-cb3e3ec512b0',
   '11cba53a-b6d2-54b6-9d16-d3f8de1094e8',
   '8f75c22a-713f-50ff-bab8-88d5da987916',
   '9b85eeed-7c7d-5273-b6ec-8071f6506273',
   'b9fa842e-7baa-5f6a-97f7-836e15b2e4df',
   '03626e46-d119-5117-a79e-3d44b416c1e0']],
 'embeddings': None,
 'documents': [['CPU中断部件是实现中断功能的软件和硬件，有如下的功能:(1) 允许和禁止中断请求(2) 保护和恢复中断现场(3) 将程序转向对应的中断服务程序(4) 在多个中断同时发生时进行判优裁决。',
   'CPU中断控制部件有哪些功能？',
   '存储器中的指令和数据都是通过二进制的形式保存的，如果直接进行取出，我们也不能判断哪些是指令，哪些是数据，但是，我们可以通过一些手段进行区分，例如取出的时机：如果是在读取指令（微程序）的时候，那么取出的二进制内容就是指令，如果是在执行指令（微程序）的时候取出的则为数据。一般的，数据和指令，都可以成为指令执行的操作对象。',
   '区分指令和数据有两种办法：(1) 通过取出时机来判断: 如果是在取指令（微程序）的时候，取出的就是指令，如果是在执行指令（微程序）的时候，取出的时数据。(2) 通过地址来源来区分：如果是由 PC（程序计数器）提供的存储单元地址，那么取出的为指令，如果是地址码提供的存储单元地址，那么取出来的是数据。',
   'A. 原码\\nB. 补码\\nC. 反码\\nD. 移码',
   'A. 反码\\nB. 补码\\nC. 原码\\nD. 移码',
   'A. 原码\\

## Custom


In [3]:
import uuid
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from chromadb import EmbeddingFunction, Embeddings, Documents
from chromadb.utils.embedding_functions import HuggingFaceEmbeddingFunction

class BAAIEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_name: str = "BAAI/bge-large-zh-v1.5"):
        self.model_name = model_name
        self.model = HuggingFaceEmbedding(model_name=model_name)
    
    def __call__(self, input: Documents) -> Embeddings:
        return self.model.get_text_embedding_batch(input)

# 初始化嵌入模型
bge_embeddings = HuggingFaceEmbedding(model_name="BAAI/bge-large-zh-v1.5")

client = chromadb.PersistentClient(path="./db_custom")

collection = client.get_or_create_collection("custom", embedding_function=BAAIEmbeddingFunction())
collection_no_custom = client.get_or_create_collection("no_custom")
# collection = client.get_or_create_collection("hf_embedding", embedding_function=HuggingFaceEmbeddingFunction(model_name="BAAI/bge-large-zh-v1.5"))

from knowledge_base.utils import load_data
from pathlib import Path
df = load_data(Path("../data/questions.csv"))

for _, row in df.iterrows():
    d = row.to_dict()
    print(d)
    collection.add(
        documents=[str(d["document"])],
        metadatas=[d],
        ids=[str(uuid.uuid4())]
    )
    collection_no_custom.add(
        documents=[str(d["document"])],
        metadatas=[d],
        ids=[str(uuid.uuid4())]
    )

2025-08-17 20:50:28 [INFO] [utils.load_data] - DataFrame loaded from: ../data/questions.csv


{'subject': '计算机组成原理', 'source': '互联网', 'type': '选择题', 'difficulty': '中等', 'options': 'A. 89.8\\nB. 211.4\\nC. 211.5\\nD. 1011111.101', 'exam_point': '进制转换', 'document': '题目：若十进制数为137.5，则其八进制数为（B）。\\nA. 89.8\\nB. 211.4\\nC. 211.5\\nD. 1011111.101\\n答案：B. 211.4', 'question': '若十进制数为137.5，则其八进制数为（）。\\nA. 89.8\\nB. 211.4\\nC. 211.5\\nD. 1011111.101', 'answer': 'B. 211.4'}
{'subject': '计算机组成原理', 'source': '互联网', 'type': '选择题', 'difficulty': '简单', 'options': 'A. 0~65536\\nB. 0~65535\\nC. -32768~32767\\nD. -32768~32768', 'exam_point': '数制表示范围', 'document': '题目：一个16位无符号二进制数的表示范围是（B）。\\nA. 0~65536\\nB. 0~65535\\nC. -32768~32767\\nD. -32768~32768\\n答案：B. 0~65535', 'question': '一个16位无符号二进制数的表示范围是（）。\\nA. 0~65536\\nB. 0~65535\\nC. -32768~32767\\nD. -32768~32768', 'answer': 'B. 0~65535'}
{'subject': '计算机组成原理', 'source': '互联网', 'type': '选择题', 'difficulty': '中等', 'options': 'A. 任何二进制整数都可以用十进制表示\\nB. 任何二进制小数都可以用十进制表示\\nC. 任何十进制整数都可以用二进制表示\\nD. 任何十进制小数都可以用二进制表示', 'exam_point': '进制转换', 'document': '题目：

In [4]:
collection.query(
    query_texts=["程序的中断方式"],
    n_results=1,
)

{'ids': [['ada0bbd4-e5cf-4cf1-965b-35f561ec4565']],
 'embeddings': None,
 'documents': [['题目：CPU中断控制部件有哪些功能？答案：CPU中断部件是实现中断功能的软件和硬件，有如下的功能:(1) 允许和禁止中断请求(2) 保护和恢复中断现场(3) 将程序转向对应的中断服务程序(4) 在多个中断同时发生时进行判优裁决。']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'options': '无',
    'exam_point': '中断机制',
    'answer': 'CPU中断部件是实现中断功能的软件和硬件，有如下的功能:(1) 允许和禁止中断请求(2) 保护和恢复中断现场(3) 将程序转向对应的中断服务程序(4) 在多个中断同时发生时进行判优裁决。',
    'source': '互联网',
    'type': '简答题',
    'difficulty': '中等',
    'document': '题目：CPU中断控制部件有哪些功能？答案：CPU中断部件是实现中断功能的软件和硬件，有如下的功能:(1) 允许和禁止中断请求(2) 保护和恢复中断现场(3) 将程序转向对应的中断服务程序(4) 在多个中断同时发生时进行判优裁决。',
    'subject': '计算机组成原理',
    'question': 'CPU中断控制部件有哪些功能？'}]],
 'distances': [[0.7982203364372253]]}

In [5]:
collection_no_custom.query(
    query_texts=["程序的中断方式"],
    n_results=3,
)

{'ids': [['cd0d000e-aa1d-45d7-b3fa-0ba334163dfd',
   '8e36f8aa-7843-475a-abcb-92a0ca62aecb',
   'ed098aa0-56c8-4974-9e53-9bbd5de68ab8']],
 'embeddings': None,
 'documents': [['题目：设机器数采用补码表示（含1位符号位），若寄存器内容为9BH，则对应的十进制数为（C）。\\nA. -27\\nB. -97\\nC. -101\\nD. 155\\n答案：C. -101',
   '题目：假定一个十进制数为-66，按补码形式存放在一个8位寄存器中，该寄存器的内容用十六进制表示为（B）。\\nA. C2H\\nB. BEH\\nC. BDH\\nD. 42H\\n答案：B. BEH',
   '题目：若十进制数为137.5，则其八进制数为（B）。\\nA. 89.8\\nB. 211.4\\nC. 211.5\\nD. 1011111.101\\n答案：B. 211.4']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'answer': 'C. -101',
    'options': 'A. -27\\nB. -97\\nC. -101\\nD. 155',
    'exam_point': '补码计算',
    'source': '互联网',
    'question': '设机器数采用补码表示（含1位符号位），若寄存器内容为9BH，则对应的十进制数为（）。\\nA. -27\\nB. -97\\nC. -101\\nD. 155',
    'document': '题目：设机器数采用补码表示（含1位符号位），若寄存器内容为9BH，则对应的十进制数为（C）。\\nA. -27\\nB. -97\\nC. -101\\nD. 155\\n答案：C. -101',
    'type': '选择题',
    'subject': '计算机组成原理',
    'difficulty': '困难'},
   {'question

In [6]:
collection_check = client.get_collection(name="custom")
collection_check.query(
    query_texts=["程序的中断方式"],
    n_results=1,
)

InvalidArgumentError: Collection expecting embedding with dimension of 1024, got 384